In [1]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd

In [2]:
PATH_DADOS = 'original_data/dados_portal_dados_abertos/'

In [3]:
def gerar_sopa():
    
    #teoricamente se trocar essa url funciona para outros conjuntos de dados
    url_conjunto = ('http://dados.prefeitura.sp.gov.br/dataset/'
                'remuneracao-servidores-prefeitura-de-sao-paulo')
    print('Fazendo primeira requisição')
    with requests.get(url_conjunto) as r:
        sopa = BeautifulSoup(r.text)
        
    return sopa

In [4]:
def pegar_itens(sopa):
    
    itens = sopa.find_all('li', {'class' : 'resource-item'})
    
    return itens

In [5]:
def get_descricao(item):
    
    desc=item.find('p', {'class' : 'description'}).text
    
    return desc.strip()

In [6]:
def get_download_link(item):
    
    menu = item.find('ul', {'class' : 'dropdown-menu'})
    link = menu.find('a', {'class' : "resource-url-analytics"})
    
    return link['href']

In [7]:
def download_data(link):
    
    with requests.get(link) as r:
        dados = r.text
    
    return dados

In [8]:
def gen_fname(desc, path_dados=PATH_DADOS):
    
    mes = desc.split(':')[-1]
    fname = mes.replace(' ', '_') + '.csv'
    
    return os.path.join(path_dados, fname)

In [9]:
def save_file(fname, dados):
    
    with open(fname, 'w') as f:
        f.write(dados)

In [10]:
def check_if_already_saved(desc):
    
    fname = gen_fname(desc)
    
    return os.path.exists(fname)

In [11]:
def download_one_item(item, link):
    
    desc = get_descricao(item)
    
    if check_if_already_saved(desc):
        print(f'Dados para {desc} já salvos')
        return
    else:
        print(f'Baixando dados para: {desc}')
        dados = download_data(link)
        fname = gen_fname(desc)
        save_file(fname, dados)
        print(f'Dados salvos para {desc} baixados')
    

In [12]:
def download_all_itens(itens=None):
    
    for item in itens:
        link = get_download_link(item)

        if link.endswith('.csv'):
            download_one_item(item, link)
    

In [13]:
def get_all_csv():
    
    sopa = gerar_sopa()
    itens = pegar_itens(sopa)
    download_all_itens(itens)

In [14]:
get_all_csv()

Fazendo primeira requisição
Dados para Mês de referência: julho de 2021 já salvos
Dados para Mês de referência: junho de 2021 já salvos
Dados para Mês de referência: maio de 2021 já salvos
Dados para Mês de referência: abril de 2021 já salvos
Dados para Mês de referência: março de 2021 já salvos
Dados para Mês de referência: fevereiro de 2021 já salvos
Dados para Mês de referência: janeiro de 2021 já salvos
Dados para Mês de referência: dezembro de 2020 já salvos
Dados para Mês de referência: novembro de 2020 já salvos
Dados para Mês de referência: outubro de 2020 já salvos
Dados para Mês de referência: setembro de 2020 já salvos
Dados para Mês de referência: agosto de 2020 já salvos
Dados para Mês de referência: julho de 2020 já salvos
Dados para Mês de referência: junho de 2020 já salvos
Dados para Mês de referência: maio de 2020 já salvos
Dados para Mês de referência: abril de 2020 já salvos
Dados para Mês de referência: março de 2020 já salvos
Dados para Mês de referência: fevereir

In [15]:
def open_all_historic_salary_data(save=True, path_dados=PATH_DADOS):
    
    files = [os.path.join(path_dados, file) for 
            file in os.listdir(path_dados) if file.endswith('.csv')]
    dfs=[]
    for file in files:
        df = pd.read_csv(file, sep=';', encoding='utf-8')
        df['filename'] = file
        dfs.append(df)
    
    df_final = pd.concat(dfs)
    
    if save:
        df_final.to_csv(os.path.join('original_data/aggregated/',
                        'historico_remuneracao_jan_2017_jul_2021.csv'),
                       sep=';', encoding='utf-8')
    
    return df_final

In [16]:
df = open_all_historic_salary_data()

In [17]:
df.head()

,Exceção,Nome completo,Cargo Base,Cargo em Comissão,Remuneração do Mês,Demais Elementos da Remuneração,Remuneração Bruta,Unidade,Tp. Log,Logadrouro,Número,Complemento,Jornada,filename,Unnamed: 0
0,NaN,ADENIZE CONRADO DOS SANTOS,,ASSESSOR I,"2.546,86","1.004,92","3.551,78",GABINETE DO PREFEITO,Viaduto,DO CHA,15,5º ANDAR,40 H,original_data/dados_portal_dados_abertos/_feve...,NaN
1,NaN,ADLA RIBEIRO DOS SANTOS,,ASSESSOR TECNICO II,"3.385,57","1.200,94","4.586,51",GABINETE DO PREFEITO,Viaduto,DO CHA,15,5º ANDAR,40 H,original_data/dados_portal_dados_abertos/_feve...,NaN
2,NaN,ADRIANA APARECIDA FERNANDES BIFULCO,,ASSESSOR TECNICO I,"3.077,73","1.032,72","4.110,45",COORDENACAO DE IMPRENSA,Viaduto,DO CHA,15,6º ANDAR,40 H,original_data/dados_portal_dados_abertos/_feve...,NaN
3,NaN,ADRIANA PETRILLI LEME DE CAMPOS,PROCURADOR DO MUNICIPIO II,ASSESSOR JURIDICO III,"32.181,49","719,98","32.901,47",ASSESSORIA JURIDICA,Viaduto,DO CHA,15,9º ANDAR,40 H,original_data/dados_portal_dados_abertos/_feve...,NaN
4,NaN,ALBERTINA FLORENTINO DE SOUZA,AGENTE DE APOIO NIVEL I,ASSESSOR TECNICO I,"2.232,93","869,32","3.102,25",ASSESSORIA TECNICO-LEGISLATIVA,Viaduto,DO CHA,15,6º ANDAR,40 H,original_data/dados_portal_dados_abertos/_feve...,NaN


In [18]:
link_dados_cadastrais = 'http://dados.prefeitura.sp.gov.br/dataset/bf5df0f4-4fb0-4a5e-b013-07d098cc7b1c/resource/17ef6f66-f1b2-4455-9b92-fb66992571a7/download/verificadoativos02-08-2021jul-2021.csv'

In [19]:
def download_dados_cadastrais(link = link_dados_cadastrais):
    
    dados = download_data(link)
    
    fname = 'original_data/aggregated/cadastro_servidores_ativos_jul_2021.csv'
    save_file(fname, dados)

In [20]:
download_dados_cadastrais()